В Google Chrome при открытии PDF через элемент <embed> браузер добавляет свою собственную панель инструментов для управления PDF. Кнопка скачивания находится именно там.

Как найти кнопку скачивания в Chrome:
Она не в вашем HTML — она добавляется браузером автоматически.

Расположение: обычно в правом верхнем углу панели инструментов Chrome для PDF.

### Блок кода для парсинга

In [11]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import os
import time
import glob
import requests
from selenium.common.exceptions import WebDriverException

# Настройка параметров загрузки
download_dir = r"C:\Users\Катерина Андреевна\code\парсинг\pdf + supplementary information"
if not os.path.exists(download_dir):
    os.makedirs(download_dir)
    print(f"Создана папка: {download_dir}")

# Конфигурация ChromeOptions
options = uc.ChromeOptions()
prefs = {
    "download.default_directory": download_dir,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "plugins.always_open_pdf_externally": True,
    "profile.default_content_settings.popups": 0,
    "profile.content_settings.exceptions.automatic_downloads.*.setting": 1
}
options.add_experimental_option("prefs", prefs)
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--disable-extensions")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# Инициализация драйвера
driver = uc.Chrome(options=options)
wait = WebDriverWait(driver, 30)

# Функция для ожидания загрузки любого нового файла
def wait_for_download(download_dir, timeout=60):
    print("Ожидание загрузки файла...")
    start_time = time.time()
    initial_files = set(glob.glob(os.path.join(download_dir, "*.*")))
    
    while time.time() - start_time < timeout:
        current_files = set(glob.glob(os.path.join(download_dir, "*.*")))
        new_files = current_files - initial_files
        if new_files:
            downloaded_file = new_files.pop()
            print(f"Файл загружен: {downloaded_file}")
            return True
        time.sleep(1)
    return False

# Открываем страницу
try:
    print("Открываем страницу статьи")
    driver.get('https://doi.org/10.1021/acscentsci.5b00399')
    
    # Даем странице полностью загрузиться
    time.sleep(5)

    # Запускаем скачивание pdf
    try:
        # Вариант 1: По классу и атрибуту title
        PDF_button = wait.until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.article__btn__secondary--pdf[title="PDF"]'))
        )
        print("Кнопка найдена по CSS селектору")
    except:
        try:
            # Вариант 2: По data-id
            PDF_button = wait.until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'a[data-id="article_header_OpenPDF"]'))
            )
        except:
            # Вариант 3: По XPath - по частичному совпадению href
            PDF_button = wait.until(
                EC.element_to_be_clickable((By.XPATH, '//a[contains(@href, "/doi/pdf/")]'))
            )
            print("Кнопка найдена по ссылке")
    
    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", PDF_button)
    time.sleep(1)
    
    # Проверяем наличие href для прямой загрузки
    pdf_href = PDF_button.get_attribute("href")
    print(f"Ссылка на PDF: {pdf_href}")
    
    if pdf_href and "/doi/pdf/" in pdf_href:
        print("Попытка скачать PDF напрямую через requests")
        try:
            # Добавляем заголовки для обхода блокировки
            headers = {
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
                "Referer": "https://pubs.acs.org/doi/10.1021/acscentsci.5b00399"
            }
            response = requests.get(pdf_href, headers=headers, stream=True)
            if response.status_code == 200:
                with open(os.path.join(download_dir, "article.pdf"), "wb") as f:
                    f.write(response.content)
                print("PDF успешно скачан через requests")
            else:
                print(f"Ошибка при скачивании PDF через requests: статус {response.status_code}")
                raise Exception("Не удалось скачать через requests")
        except Exception as e:
            print(f"Ошибка при скачивании через requests: {e}")
            # Переходим к клику по кнопке
            print("Попытка скачать PDF через клик по кнопке")
            try:
                PDF_button.click()
            except:
                driver.execute_script("arguments[0].click();", PDF_button)
            print("Кнопка Open PDF нажата")
            
            # Проверка открытия новой вкладки
            time.sleep(3)
            
            if len(driver.window_handles) > 1:
                driver.switch_to.window(driver.window_handles[1])
                print("Переключились на новую вкладку")
                
                try:
                    embed_element = driver.find_element(By.TAG_NAME, "embed")
                    pdf_url = driver.execute_script("return arguments[0].getAttribute('src');", embed_element)
                    
                    if pdf_url and pdf_url != "about:blank":
                        response = requests.get(pdf_url, headers=headers, stream=True)
                        if response.status_code == 200:
                            with open(os.path.join(download_dir, "article.pdf"), "wb") as f:
                                f.write(response.content)
                            print("Файл скачан через requests из новой вкладки")
                        else:
                            print(f"Ошибка при скачивании из новой вкладки: статус {response.status_code}")
                    else:
                        print("PDF URL не найден в новой вкладке")
                except WebDriverException as e:
                    print(f"Ошибка при поиске embed элемента: {e}")
                finally:
                    driver.close()
                    driver.switch_to.window(driver.window_handles[0])
            else:
                print("Новая вкладка не открылась, работаем в текущей")
                if wait_for_download(download_dir):
                    print("PDF успешно скачан через браузер")
                else:
                    print("Не удалось скачать PDF: файл не найден")
    else:
        print("Прямая ссылка на PDF недоступна, пробуем клик")
        try:
            PDF_button.click()
        except:
            driver.execute_script("arguments[0].click();", PDF_button)
        print("Кнопка Open PDF нажата")
        
        time.sleep(3)
        
        if len(driver.window_handles) > 1:
            driver.switch_to.window(driver.window_handles[1])
            print("Переключились на новую вкладку")
            
            try:
                embed_element = driver.find_element(By.TAG_NAME, "embed")
                pdf_url = driver.execute_script("return arguments[0].getAttribute('src');", embed_element)
                
                if pdf_url and pdf_url != "about:blank":
                    headers = {
                        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
                        "Referer": "https://pubs.acs.org/doi/10.1021/acscentsci.5b00399"
                    }
                    response = requests.get(pdf_url, headers=headers, stream=True)
                    if response.status_code == 200:
                        with open(os.path.join(download_dir, "article.pdf"), "wb") as f:
                            f.write(response.content)
                        print("Файл скачан через requests из новой вкладки")
                    else:
                        print(f"Ошибка при скачивании из новой вкладки: статус {response.status_code}")
                else:
                    print("PDF URL не найден в новой вкладке")
            except WebDriverException as e:
                print(f"Ошибка при поиске embed элемента: {e}")
            finally:
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
        else:
            print("Новая вкладка не открылась, работаем в текущей")
            if wait_for_download(download_dir):
                print("PDF успешно скачан через браузер")
            else:
                print("Не удалось скачать PDF: файл не найден")

    # Скачиваем Supporting Information 
    try:
        print("Переходим к Supporting Information")
        driver.switch_to.window(driver.window_handles[0])  # Возвращаемся на основную вкладку
        sup_button = wait.until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.article__btn__secondary.internalNav[title="Supporting Information"]'))
        )
        
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", sup_button)
        time.sleep(1)
        
        print("Нажатие ссылки Supporting Information")
        try:
            sup_button.click()
        except:
            driver.execute_script("arguments[0].click();", sup_button)
        print("Кнопка Supporting Information нажата")
        
        # Ожидание загрузки страницы Supporting Information
        time.sleep(3)
        
        # Находим и нажимаем кнопку Download
        try:
            download_button = wait.until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.frontend-filesViewer-inlineMode-downloadMenu-trigger-module__downloadButton--UsBvq'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", download_button)
            time.sleep(1)
            
            print("Нажатие кнопки Download для Supporting Information")
            try:
                download_button.click()
            except:
                driver.execute_script("arguments[0].click();", download_button)
            print("Кнопка Download нажата")
            
            # Ожидание загрузки файла
            if wait_for_download(download_dir):
                print("Supporting Information успешно скачан")
            else:
                print("Не удалось скачать Supporting Information: файл не найден")
                
        except Exception as e:
            print(f"Ошибка при поиске кнопки Download: {str(e)}")
            
    except Exception as e:
        print(f"Не удалось скачать Supporting Information: {str(e)}")

finally:
    # Закрываем драйвер безопасно
    try:
        driver.quit()
        print("Браузер закрыт")
    except WebDriverException:
        print("Браузер уже закрыт или произошла ошибка при закрытии")

Открываем страницу статьи
Кнопка найдена по CSS селектору
Ссылка на PDF: https://pubs.acs.org/doi/pdf/10.1021/acscentsci.5b00399?ref=article_openPDF
Попытка скачать PDF напрямую через requests
Ошибка при скачивании PDF через requests: статус 403
Ошибка при скачивании через requests: Не удалось скачать через requests
Попытка скачать PDF через клик по кнопке
Кнопка Open PDF нажата
Новая вкладка не открылась, работаем в текущей
Ожидание загрузки файла...
Не удалось скачать PDF: файл не найден
Переходим к Supporting Information
Нажатие ссылки Supporting Information
Кнопка Supporting Information нажата
Нажатие кнопки Download для Supporting Information
Кнопка Download нажата
Ожидание загрузки файла...
Файл загружен: C:\Users\Катерина Андреевна\code\парсинг\pdf + supplementary information\oc5b00399_si_001.pdf
Supporting Information успешно скачан
Браузер закрыт
